In [3]:
import pandas as pd
import os
import glob
from tasks import compute_index
from tasks.post_process import compare_2020_2019_data_report

In [4]:
info_df = compare_2020_2019_data_report()

In [6]:
info_df.to_csv('data/results/2020_2019_report.csv')